In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


#### IMPORT THE LIB

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2024-07-02 09:27:42.954088: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 09:27:42.954233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 09:27:43.092697: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


#### DATA COLLECTION

In [26]:
train=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

#### DATA CLEANING

In [27]:
train=train.dropna(axis=1)
test=test.dropna(axis=1)

In [28]:
set(train.columns)-set(test.columns)

{'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtUnfSF',
 'Exterior1st',
 'Exterior2nd',
 'Functional',
 'GarageArea',
 'GarageCars',
 'KitchenQual',
 'MSZoning',
 'SalePrice',
 'SaleType',
 'TotalBsmtSF',
 'Utilities'}

In [29]:
train=train.drop(['BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtUnfSF',
 'Exterior1st',
 'Exterior2nd',
 'Functional',
 'GarageArea',
 'GarageCars',
 'KitchenQual',
 'MSZoning',
 'SaleType',
 'TotalBsmtSF',
 'Utilities'],axis=1)

In [30]:
set(test.columns)-set(train.columns)

{'Electrical'}

In [31]:
test=test.drop(['Electrical'],axis=1)

In [32]:
train.shape,test.shape

((1460, 47), (1459, 46))

In [33]:
train[train.isnull()==True].count(),test[test.isnull()==True].count()

(Id               0
 MSSubClass       0
 LotArea          0
 Street           0
 LotShape         0
 LandContour      0
 LotConfig        0
 LandSlope        0
 Neighborhood     0
 Condition1       0
 Condition2       0
 BldgType         0
 HouseStyle       0
 OverallQual      0
 OverallCond      0
 YearBuilt        0
 YearRemodAdd     0
 RoofStyle        0
 RoofMatl         0
 ExterQual        0
 ExterCond        0
 Foundation       0
 Heating          0
 HeatingQC        0
 CentralAir       0
 1stFlrSF         0
 2ndFlrSF         0
 LowQualFinSF     0
 GrLivArea        0
 FullBath         0
 HalfBath         0
 BedroomAbvGr     0
 KitchenAbvGr     0
 TotRmsAbvGrd     0
 Fireplaces       0
 PavedDrive       0
 WoodDeckSF       0
 OpenPorchSF      0
 EnclosedPorch    0
 3SsnPorch        0
 ScreenPorch      0
 PoolArea         0
 MiscVal          0
 MoSold           0
 YrSold           0
 SaleCondition    0
 SalePrice        0
 dtype: int64,
 Id               0
 MSSubClass       0
 LotA

ABOVE NO MISSING VALUES IN TRAIN AND TEST DATA.

In [35]:
train.columns,test.columns

(Index(['Id', 'MSSubClass', 'LotArea', 'Street', 'LotShape', 'LandContour',
        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
        'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond',
        'Foundation', 'Heating', 'HeatingQC', 'CentralAir', '1stFlrSF',
        '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath',
        'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
        'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
        'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold',
        'SaleCondition', 'SalePrice'],
       dtype='object'),
 Index(['Id', 'MSSubClass', 'LotArea', 'Street', 'LotShape', 'LandContour',
        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
        'YearRemodAdd', 'RoofStyle', '

In [36]:
y=train['SalePrice']
X=train.drop(['SalePrice'],axis=1)

In [37]:
X.shape,test.shape

((1460, 46), (1459, 46))

#### CATEGORICAL AND NUMERICAL FEATURES

In [38]:
cat_cols=[col for col in X.columns if X[col].dtype=='object']
num_cols=[col for col in X.columns if X[col].dtype!='object']

In [39]:
cat_cols

['Street',
 'LotShape',
 'LandContour',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'PavedDrive',
 'SaleCondition']

#### LABEL ENCODING
Label encoding assigns a unique integer to each category.Unlike,ordinal encoding it does not imply any specific order among the categories

In [44]:
from sklearn.preprocessing import LabelEncoder

encode=LabelEncoder()

for i in X.columns:
    X[i]=encode.fit_transform(X[i])
    test[i]=encode.fit_transform(test[i])
    

In [54]:
X

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
0,0,5,327,1,3,3,4,0,5,2,...,0,49,0,0,0,0,0,1,2,4
1,1,0,498,1,3,3,2,0,24,1,...,187,0,0,0,0,0,0,4,1,4
2,2,5,702,1,0,3,4,0,5,2,...,0,30,0,0,0,0,0,8,2,4
3,3,6,489,1,0,3,0,0,6,2,...,0,24,108,0,0,0,0,1,0,0
4,4,5,925,1,0,3,2,0,15,2,...,118,70,0,0,0,0,0,11,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,5,267,1,3,3,4,0,8,2,...,0,28,0,0,0,0,0,7,1,4
1456,1456,0,866,1,3,3,4,0,14,2,...,214,0,0,0,0,0,0,1,4,4
1457,1457,6,415,1,3,3,4,0,6,2,...,0,48,0,0,0,0,17,4,4,4
1458,1458,0,505,1,3,3,4,0,12,2,...,224,0,38,0,0,0,0,3,4,4


In [55]:
X.describe()

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,729.500000,4.166438,492.102740,0.995890,1.942466,2.777397,3.019178,0.062329,12.251370,2.031507,...,53.979452,35.636986,8.205479,0.158904,2.873973,0.019178,0.290411,5.321918,1.815753,3.770548
std,421.610009,4.161951,300.719399,0.063996,1.409156,0.707666,1.622634,0.276232,6.013735,0.868515,...,72.089748,47.892383,23.302645,1.388076,11.273071,0.309173,1.793077,2.703626,1.328095,1.100854
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,364.750000,0.000000,229.750000,1.000000,0.000000,3.000000,2.000000,0.000000,7.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,1.000000,4.000000
50%,729.500000,4.000000,476.500000,1.000000,3.000000,3.000000,4.000000,0.000000,12.000000,2.000000,...,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2.000000,4.000000
75%,1094.250000,6.000000,737.250000,1.000000,3.000000,3.000000,4.000000,0.000000,17.000000,2.000000,...,97.000000,56.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,3.000000,4.000000
max,1459.000000,14.000000,1072.000000,1.000000,3.000000,3.000000,4.000000,2.000000,24.000000,8.000000,...,273.000000,201.000000,119.000000,19.000000,75.000000,7.000000,20.000000,11.000000,4.000000,5.000000


#### MUTUAL INFORMATION
Mutual Info is a measure from the information theory that quantifies the amount of information obtained about one random variable through another random variable.
* `HIGH MI` - knowing the value one var provides lot of info about the other var.
* `LOW MI` - Knowing the value one var provides less info about the other var.

In [58]:
from sklearn.feature_selection import mutual_info_regression

#Calculate MI between feature and the target var
mi=mutual_info_regression(X,y)

#Create a dataframe for easier interpreataion
mi_data=pd.DataFrame({'features':X.columns,
                     'MI':mi})
mi_data=mi_data.sort_values(by='MI',ascending=False)
print(mi_data)

         features        MI
13    OverallQual  0.530281
28      GrLivArea  0.347492
19      ExterQual  0.334658
15      YearBuilt  0.287536
29       FullBath  0.244877
25       1stFlrSF  0.217981
33   TotRmsAbvGrd  0.210002
16   YearRemodAdd  0.189489
21     Foundation  0.179543
34     Fireplaces  0.171337
26       2ndFlrSF  0.143638
23      HeatingQC  0.118317
14    OverallCond  0.115296
2         LotArea  0.107654
8    Neighborhood  0.107520
24     CentralAir  0.090599
45  SaleCondition  0.088196
30       HalfBath  0.086773
1      MSSubClass  0.083352
4        LotShape  0.080751
12     HouseStyle  0.077904
37    OpenPorchSF  0.069610
31   BedroomAbvGr  0.062219
36     WoodDeckSF  0.050564
9      Condition1  0.042944
5     LandContour  0.041223
11       BldgType  0.037940
35     PavedDrive  0.034155
20      ExterCond  0.032333
38  EnclosedPorch  0.027360
40    ScreenPorch  0.024353
32   KitchenAbvGr  0.016816
17      RoofStyle  0.015159
27   LowQualFinSF  0.007950
6       LotConfig  0

In [66]:
feature_list=[]
feature_list=mi_data[mi_data['MI']>0.01]['features']
print(feature_list)

13      OverallQual
28        GrLivArea
19        ExterQual
15        YearBuilt
29         FullBath
25         1stFlrSF
33     TotRmsAbvGrd
16     YearRemodAdd
21       Foundation
34       Fireplaces
26         2ndFlrSF
23        HeatingQC
14      OverallCond
2           LotArea
8      Neighborhood
24       CentralAir
45    SaleCondition
30         HalfBath
1        MSSubClass
4          LotShape
12       HouseStyle
37      OpenPorchSF
31     BedroomAbvGr
36       WoodDeckSF
9        Condition1
5       LandContour
11         BldgType
35       PavedDrive
20        ExterCond
38    EnclosedPorch
40      ScreenPorch
32     KitchenAbvGr
17        RoofStyle
Name: features, dtype: object


In [67]:
set(X.columns)-set(feature_list)

{'3SsnPorch',
 'Condition2',
 'Heating',
 'Id',
 'LandSlope',
 'LotConfig',
 'LowQualFinSF',
 'MiscVal',
 'MoSold',
 'PoolArea',
 'RoofMatl',
 'Street',
 'YrSold'}

In [68]:
X=X.drop(['3SsnPorch',
 'Condition2',
 'Heating',
 'Id',
 'LandSlope',
 'LotConfig',
 'LowQualFinSF',
 'MiscVal',
 'MoSold',
 'PoolArea',
 'RoofMatl',
 'Street',
 'YrSold'],axis=1)
test=test.drop(['3SsnPorch',
 'Condition2',
 'Heating',
 'Id',
 'LandSlope',
 'LotConfig',
 'LowQualFinSF',
 'MiscVal',
 'MoSold',
 'PoolArea',
 'RoofMatl',
 'Street',
 'YrSold'],axis=1)

#### MINMAXSCALER (NORMALISATION)

In [70]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
X_scale=scaler.fit_transform(X)
test_scale=scaler.fit_transform(test)

#### DATA SPLITTING

In [72]:
from sklearn.model_selection import train_test_split

X_train,X_valid,y_train,y_valid=train_test_split(X_scale,y,
                                                 test_size=0.2,random_state=1)

#### MODEL (NEURAL NETWORKS)

In [78]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

#### NUMBER OF DENSE LAYERS/NUMBER OF NEURONS HELPS TO REDUCE MAE

In [123]:
tf.random_seed=1
model_1=Sequential([
    Dense(128,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(8,activation='relu'),
    Dense(4,activation='relu'),
    Dense(2,activation='relu'),
    Dense(1)
])

model_1.compile(loss='mean_absolute_error',
               optimizer='adam',
               metrics=['mae'])

history_1=model_1.fit(X_train,y_train,epochs=10,validation_data=(X_valid,y_valid))

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 181ms/step - loss: 182195.2969 - mae: 182195.2969 - val_loss: 175772.5625 - val_mae: 175772.5625
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 181249.7188 - mae: 181249.7188 - val_loss: 175772.5312 - val_mae: 175772.5312
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 185716.8750 - mae: 185716.8750 - val_loss: 175772.5000 - val_mae: 175772.5000
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 181988.8281 - mae: 181988.8281 - val_loss: 175772.4531 - val_mae: 175772.4531
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 185123.4844 - mae: 185123.4844 - val_loss: 175772.4062 - val_mae: 175772.4062
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 184388.5625 - mae: 184388.5625 - val_loss: 175772.3906 - val_mae: 175772.3906
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 180976.4219 - mae: 180976.4219 - val_loss: 175772.3750 - val_mae: 175772.3750
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss

In [124]:
model_1.evaluate(X_valid,y_valid)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 177512.0938 - mae: 177512.0938 


[175772.25, 175772.25]

In [125]:
model_1.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_72 (Dense)                │ (None, 128)            │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_73 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_74 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_75 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_76 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_77 (Dense)                │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_78 (Dense)                │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_79 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,205 (180.49 KB)

 Trainable params: 15,401 (60.16 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 30,804 (120.33 KB)

#### CONLUSION 
`NEURAL NETWORKS` ARE INITIALISE WITH `RANDOM WEIGHTS`.EVEN THOUGH THE ARCHITECTURE IS IDENTICAL,DIFFERENT INITIALISATIONS CAN LEAD TO DIFFERENT `CONVERGENCE` PATHS AND FINAL RESULTS.

#### OPITIMIZER

In [155]:
tf.random_seed=1
model_2=Sequential([
        Dense(128,activation='relu',input_shape=(X_train.shape[1],)),
        Dense(64,activation='relu'),
        Dense(32,activation='relu'),
        Dense(16,activation='relu'),
        Dense(8,activation='relu'),
        Dense(4,activation='relu'),
        Dense(2,activation='relu'),
        Dense(1)
])

model_2.compile(loss='mean_absolute_error',
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                   metrics=['mae'])

hist_2=model_2.fit(X_train,y_train,epochs=100,batch_size=32,validation_data=(X_valid,y_valid))

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 179ms/step - loss: 181689.9688 - mae: 181689.9688 - val_loss: 175772.3750 - val_mae: 175772.3750
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 182000.6875 - mae: 182000.6875 - val_loss: 175761.7812 - val_mae: 175761.7812
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 182079.0938 - mae: 182079.0938 - val_loss: 175534.2969 - val_mae: 175534.2969
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 178732.2500 - mae: 178732.2500 - val_loss: 172500.0625 - val_mae: 172500.0625
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 175742.1562 - mae: 175742.1562 - val_loss: 145416.3438 - val_mae: 145416.3438
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 124233.4609 - mae: 124233.4609 - val_loss: 50340.2344 - val_mae: 50340.2344
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 47787.2812 - mae: 47787.2812 - val_loss: 42125.7656 - val_mae: 42125.7656
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

In [156]:
model_2.evaluate(X_valid,y_valid)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20283.2598 - mae: 20283.2598 


[21056.6015625, 21056.6015625]

#### CONCLUSION 
WE HAVE RECEIVED A LOW MAE:19000.
WE APPLIED ALL KNOW METHODS TILL NOW MY OTHERS NOTEBOOKS [](http://)